<a href="https://colab.research.google.com/github/sinmanjessak/AnyBlog.com/blob/master/Hybrid_GANVAE_Full_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hybrid VAE + WGAN‑GP (Final) — Cross‑Domain Time‑Series (ED + Finance)

This is the **final, robust implementation**:
- Trains **per‑domain baselines** (VAE, WGAN‑GP) on ED & Finance
- Trains **Hybrid**: shared VAE encoder → conditional WGAN‑GP + domain‑adversarial alignment
- Evaluates **quality** (ACF, PSD, DTW) and **utility** (forecasting RMSE/SMAPE)  
- No external installs needed (custom DTW included)

> Put these files next to the notebook: `ED_admission.csv`, `EUR_USD.csv`, `XAU_USD.csv`. The notebook will create `finance.csv` automatically from percentage returns.


In [7]:

import os, random, json, math, gc, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# ----------------
# Configuration
# ----------------
ED_FILE = "ED_admission.csv"
EUR_FILE = "EUR_USD.csv"
XAU_FILE = "XAU_USD.csv"
FIN_MERGED = "finance.csv"

SEQ_LEN   = 60
BATCH     = 64
# Set solid defaults. You can reduce if compute is limited.
EPOCHS_BASE   = 30   # per-domain VAE & WGAN-GP
EPOCHS_HYBRID = 60   # joint hybrid training

LR       = 2e-4
BETAS    = (0.5, 0.9)
N_CRITIC = 5
LAMBDA_GP = 10.0

NOISE_DIM = 16
Z_SHARED  = 32
Z_PRIV    = 8
GEN_H = 96
CRIT_H = 96
VAE_H = 128

FORECAST_HORIZON = 1
LSTM_H = 96


Device: cpu


In [8]:
# -------------
# Data helpers
# -------------
import os
import numpy as np                # ✅ you were missing this
import pandas as pd               # already needed

# Configuration
ED_FILE = "ED_admission.csv"
EUR_FILE = "EUR_USD.csv"
XAU_FILE = "XAU_USD.csv"
FIN_MERGED = "finance.csv"
SEQ_LEN = 60

def read_single_series_csv(path):
    df = pd.read_csv(path)
    # try common columns
    for c in ['value','close','Close','COUNT','count','y','price','Price']:
        if c in df.columns:
            ser = pd.to_numeric(df[c], errors='coerce').dropna()
            return ser.values.astype(float)
    # else last column (coerce)
    ser = pd.to_numeric(df.iloc[:, -1], errors='coerce').dropna()
    return ser.values.astype(float)

def _normalize_date_cols(df: pd.DataFrame):
    for dcol in ['date','Date','timestamp','Timestamp','datetime','Datetime']:
        if dcol in df.columns:
            df[dcol] = pd.to_datetime(df[dcol], errors='coerce')
    return df

def _to_numeric_series(s: pd.Series):
    # remove thousands separators/commas then to numeric
    return pd.to_numeric(s.astype(str).str.replace(',', '', regex=False), errors='coerce')

def build_finance_csv(eur_path, xau_path, out_path="finance.csv"):
    """
    Builds a stationary, date-aligned finance.csv from EUR/USD and XAU/USD series.
    Each row = aligned daily return across both assets.
    """
    if not (os.path.exists(eur_path) and os.path.exists(xau_path)):
        print("[WARN] EUR or XAU not found; skipping finance merge.")
        return None

    eur = pd.read_csv(eur_path)
    xau = pd.read_csv(xau_path)
    eur = _normalize_date_cols(eur)
    xau = _normalize_date_cols(xau)

    def detect_price(df):
        for c in ['close','Close','adj_close','Adj Close','value','price','Price','y','COUNT','count']:
            if c in df.columns:
                return c
        # fallback: last numeric-ish column
        for c in df.columns[::-1]:
            if np.issubdtype(df[c].dtype, np.number):
                return c
        return df.columns[-1]  # will be coerced later

    eur_col = detect_price(eur)
    xau_col = detect_price(xau)

    # Align by date if both have it
    date_col = None
    for c in ['date','Date','timestamp','Timestamp','datetime','Datetime']:
        if c in eur.columns and c in xau.columns:
            date_col = c
            break

    if date_col:
        eur_use = eur[[date_col, eur_col]].rename(columns={eur_col: "EUR_USD"})
        xau_use = xau[[date_col, xau_col]].rename(columns={xau_col: "XAU_USD"})
        eur_use["EUR_USD"] = _to_numeric_series(eur_use["EUR_USD"])
        xau_use["XAU_USD"] = _to_numeric_series(xau_use["XAU_USD"])
        eur_use = eur_use.dropna()
        xau_use = xau_use.dropna()
        df = pd.merge(eur_use, xau_use, on=date_col, how='inner').sort_values(by=date_col).set_index(date_col)
    else:
        eur_vals = _to_numeric_series(eur[eur_col]).dropna().values
        xau_vals = _to_numeric_series(xau[xau_col]).dropna().values
        n = min(len(eur_vals), len(xau_vals))
        df = pd.DataFrame({
            "EUR_USD": eur_vals[-n:],
            "XAU_USD": xau_vals[-n:]
        })

    # Debug (optional)
    # print("DataFrame before pct_change:\n", df.head())
    # print("dtypes:", df.dtypes)

    # Convert to percent returns (stationary)
    df_ret = df.pct_change().replace([np.inf, -np.inf], np.nan).dropna()

    # Save
    df_ret.to_csv(out_path, index=bool(date_col))
    print(f"✅ Saved {out_path} — shape: {df_ret.shape}, columns: {list(df_ret.columns)}")
    # print(df_ret.head())
    return out_path

def zscore(x):
    m = x.mean(); s = x.std() + 1e-8
    return (x - m)/s, m, s

def create_windows(arr, L):
    if len(arr) < L: return np.empty((0, L))
    out = [arr[i:i+L] for i in range(len(arr)-L+1)]
    return np.stack(out)

# Build (force rebuild if you want to overwrite an old file)
if not os.path.exists(FIN_MERGED):
    build_finance_csv(EUR_FILE, XAU_FILE, FIN_MERGED)

# Load ED
ed_raw = read_single_series_csv(ED_FILE)
ed_norm, ed_mean, ed_std = zscore(ed_raw)
ed_seq = create_windows(ed_norm, SEQ_LEN)

# Load Finance (composite of EUR & XAU returns)
if os.path.exists(FIN_MERGED):
    fin_df = pd.read_csv(FIN_MERGED)
    # If a Date column is present from the index, drop it for mean()
    fin_df = fin_df.drop(columns=[c for c in fin_df.columns if c.lower() in ("date","timestamp","datetime")], errors='ignore')
    fin_raw = fin_df.mean(axis=1).values.astype(float)
    fin_norm, fin_mean, fin_std = zscore(fin_raw)
    fin_seq = create_windows(fin_norm, SEQ_LEN)
else:
    fin_seq = np.empty((0, SEQ_LEN))

print("ED windows:", ed_seq.shape, " Finance windows:", fin_seq.shape)
assert len(ed_seq) >= 200, "ED dataset too small for stable training. Provide more points."


ED windows: (143221, 60)  Finance windows: (0, 60)


In [9]:

# ----------------
# Torch datasets
# ----------------
class SeqDS(Dataset):
    def __init__(self, seqs, domain_id):
        self.x = torch.tensor(seqs, dtype=torch.float32).unsqueeze(-1)
        self.d = torch.full((len(self.x),), int(domain_id), dtype=torch.long)
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return self.x[i], self.d[i]

ed_ds = SeqDS(ed_seq, 0)
ed_val_cut = max(1, int(0.1*len(ed_ds)))
ed_train = torch.utils.data.Subset(ed_ds, range(len(ed_ds)-ed_val_cut))
ed_val   = torch.utils.data.Subset(ed_ds, range(len(ed_ds)-ed_val_cut, len(ed_ds)))

if len(fin_seq) > 0:
    fin_train = SeqDS(fin_seq, 1)
else:
    fin_train = None

print("ED train/val:", len(ed_train), len(ed_val), "| FIN train:", 0 if fin_train is None else len(fin_train))


ED train/val: 128899 14322 | FIN train: 0


In [10]:

# ----------------
# VAE
# ----------------
class VAE(nn.Module):
    def __init__(self, in_dim=1, h=128, z=32):
        super().__init__()
        self.enc = nn.LSTM(in_dim, h, batch_first=True)
        self.mu   = nn.Linear(h, z)
        self.logv = nn.Linear(h, z)
        self.dec = nn.LSTM(z, h, batch_first=True)
        self.out = nn.Linear(h, in_dim)
        self.z = z
    def encode(self, x):
        _, (h, _) = self.enc(x); h = h[-1]
        return self.mu(h), self.logv(h)
    def reparam(self, mu, logv):
        std = torch.exp(0.5*logv)
        return mu + std*torch.randn_like(std)
    def decode(self, z, seq_len):
        zseq = z.unsqueeze(1).repeat(1, seq_len, 1)
        y,_ = self.dec(zseq)
        return self.out(y)
    def forward(self, x):
        mu, logv = self.encode(x)
        z = self.reparam(mu, logv)
        recon = self.decode(z, x.size(1))
        return recon, mu, logv

def vae_loss(recon, x, mu, logv, klw=1e-3):
    rec = nn.MSELoss()(recon, x)
    kld = -0.5*torch.mean(1 + logv - mu.pow(2) - logv.exp())
    return rec + klw*kld, rec.item(), kld.item()

def train_vae(ds, epochs):
    vae = VAE(1, VAE_H, Z_SHARED).to(DEVICE)
    opt = optim.Adam(vae.parameters(), lr=1e-3)
    loader = DataLoader(ds, batch_size=BATCH, shuffle=True, drop_last=True)
    for ep in range(1, epochs+1):
        s=0
        for xb,_ in loader:
            xb = xb.to(DEVICE)
            recon, mu, logv = vae(xb)
            loss, rec, kld = vae_loss(recon, xb, mu, logv, klw=5e-4 if ep>10 else 1e-4)
            opt.zero_grad(); loss.backward()
            nn.utils.clip_grad_norm_(vae.parameters(), 1.0)
            opt.step(); s+=loss.item()
        if ep%5==0 or ep==1:
            print(f"[VAE] {ep}/{epochs} loss={s/len(loader):.4f}")
    return vae


In [11]:

# ----------------
# WGAN‑GP (per domain)
# ----------------
class Generator(nn.Module):
    def __init__(self, noise_dim, z_shared, z_priv, out_dim=1, h=96, conditional=True):
        super().__init__()
        self.conditional = conditional
        in_dim = noise_dim + z_shared + (z_priv if conditional else 0) + (1 if conditional else 0)
        self.lstm = nn.LSTM(in_dim, h, batch_first=True)
        self.out = nn.Sequential(nn.Linear(h, out_dim))
    def forward(self, noise, z_shared, domain, z_priv=None):
        B,T,_ = noise.shape
        feats = [noise, z_shared.unsqueeze(1).repeat(1,T,1)]
        if self.conditional:
            if z_priv is None: z_priv = torch.zeros(B, Z_PRIV, device=noise.device)
            feats += [z_priv.unsqueeze(1).repeat(1,T,1),
                      domain.view(B,1,1).float().repeat(1,T,1)]
        x = torch.cat(feats, dim=2)
        y,_ = self.lstm(x)
        return self.out(y)

class Critic(nn.Module):
    def __init__(self, in_dim=1, h=96, conditional=True):
        super().__init__()
        self.conditional = conditional
        self.lstm = nn.LSTM(in_dim + (1 if conditional else 0), h, batch_first=True)
        self.fc = nn.Linear(h, 1)
    def forward(self, seq, domain):
        B,T,C = seq.shape
        if self.conditional:
            seq = torch.cat([seq, domain.view(B,1,1).float().repeat(1,T,1)], dim=2)
        _,(h,_) = self.lstm(seq)
        return self.fc(h[-1])

def gradient_penalty(critic, real, fake, domain):
    B = real.size(0)
    alpha = torch.rand(B,1,1, device=real.device).expand_as(real)
    inter = alpha*real + (1-alpha)*fake
    inter.requires_grad_(True)
    pred = critic(inter, domain)
    grad = torch.autograd.grad(pred, inter, torch.ones_like(pred), create_graph=True, retain_graph=True)[0]
    grad = grad.view(B, -1)
    return ((grad.norm(2, dim=1) - 1.0)**2).mean()


In [12]:

def train_wgan_gp(ds, epochs, conditional=False, domain_id=0):
    gen = Generator(NOISE_DIM, Z_SHARED, Z_PRIV, 1, GEN_H, conditional=conditional).to(DEVICE)
    crit= Critic(1, CRIT_H, conditional=conditional).to(DEVICE)
    gopt= optim.Adam(gen.parameters(), lr=LR, betas=BETAS)
    copt= optim.Adam(crit.parameters(), lr=LR, betas=BETAS)
    loader = DataLoader(ds, batch_size=BATCH, shuffle=True, drop_last=True)
    for ep in range(1, epochs+1):
        gl=cl=0.0
        for (x,d) in loader:
            x=x.to(DEVICE); d=d.to(DEVICE)
            B = x.size(0)
            # Train critic
            for _ in range(N_CRITIC):
                z = torch.randn(B, SEQ_LEN, NOISE_DIM, device=DEVICE)
                z_sh = torch.randn(B, Z_SHARED, device=DEVICE)
                z_pr = torch.randn(B, Z_PRIV, device=DEVICE) if conditional else None
                fake = gen(z, z_sh, d, z_pr)
                r = crit(x, d if conditional else d*0)
                f = crit(fake.detach(), d if conditional else d*0)
                wass = f.mean() - r.mean()
                gp = gradient_penalty(crit, x, fake.detach(), d if conditional else d*0)
                loss_d = wass + LAMBDA_GP*gp
                copt.zero_grad(); loss_d.backward()
                nn.utils.clip_grad_norm_(crit.parameters(), 1.0)
                copt.step(); cl += loss_d.item()
            # Train generator
            z = torch.randn(B, SEQ_LEN, NOISE_DIM, device=DEVICE)
            z_sh = torch.randn(B, Z_SHARED, device=DEVICE)
            z_pr = torch.randn(B, Z_PRIV, device=DEVICE) if conditional else None
            fake = gen(z, z_sh, d, z_pr)
            g_loss = -crit(fake, d if conditional else d*0).mean()
            gopt.zero_grad(); g_loss.backward()
            nn.utils.clip_grad_norm_(gen.parameters(), 1.0)
            gopt.step(); gl += g_loss.item()
        if ep%5==0 or ep==1:
            print(f"[WGAN] dom={domain_id} {ep}/{epochs}  D={cl/len(loader):.3f}  G={gl/len(loader):.3f}")
    return gen, crit


In [ ]:

# ----------------
# Train baselines
# ----------------
ed_vae = train_vae(ed_train, EPOCHS_BASE)
if fin_train is not None:
    fin_vae = train_vae(fin_train, max(10, EPOCHS_BASE//2))
else:
    fin_vae = None

ed_gen_b, ed_crit_b = train_wgan_gp(ed_train, EPOCHS_BASE, conditional=False, domain_id=0)
if fin_train is not None:
    fin_gen_b, fin_crit_b = train_wgan_gp(fin_train, max(10, EPOCHS_BASE//2), conditional=False, domain_id=1)
else:
    fin_gen_b = fin_crit_b = None


In [ ]:

# ----------------
# Hybrid (VAE encoder + conditional WGAN‑GP + domain GRL)
# ----------------
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambd): ctx.lambd = lambd; return x.view_as(x)
    @staticmethod
    def backward(ctx, g): return -ctx.lambd * g, None

class DomainHead(nn.Module):
    def __init__(self, zdim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(zdim, 64), nn.ReLU(), nn.Linear(64,2))
    def forward(self, z, lambd=1.0): return self.net(GradReverse.apply(z, lambd))

# unified dataset
if fin_train is not None:
    joint_x = np.concatenate([ed_seq, fin_seq], axis=0)
    joint_d = np.concatenate([np.zeros(len(ed_seq)), np.ones(len(fin_seq))]).astype(int)
else:
    joint_x = ed_seq; joint_d = np.zeros(len(ed_seq)).astype(int)

class JointDS(Dataset):
    def __init__(self, x, d):
        self.x = torch.tensor(x, dtype=torch.float32).unsqueeze(-1)
        self.d = torch.tensor(d, dtype=torch.long)
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return self.x[i], self.d[i]

jloader = DataLoader(JointDS(joint_x, joint_d), batch_size=BATCH, shuffle=True, drop_last=True)

hyb_gen  = Generator(NOISE_DIM, Z_SHARED, Z_PRIV, 1, GEN_H, conditional=True).to(DEVICE)
hyb_crit = Critic(1, CRIT_H, conditional=True).to(DEVICE)
hyb_dom  = DomainHead(Z_SHARED).to(DEVICE)

g_opt = optim.Adam(hyb_gen.parameters(),  lr=LR, betas=BETAS)
d_opt = optim.Adam(hyb_crit.parameters(), lr=LR, betas=BETAS)
dom_opt = optim.Adam(hyb_dom.parameters(), lr=LR, betas=BETAS)

def train_hybrid(epochs):
    for ep in range(1, epochs+1):
        dsum=gsum=ds=0.0
        for xb,db in jloader:
            xb=xb.to(DEVICE); db=db.to(DEVICE); B=xb.size(0)
            # get shared z from the appropriate VAE encoder (randomly choose domain's VAE for variety if both exist)
            with torch.no_grad():
                if fin_vae is not None and torch.rand(1).item()<0.5:
                    mu, logv = fin_vae.encode(xb)
                    z_shared = fin_vae.reparam(mu, logv)
                else:
                    mu, logv = ed_vae.encode(xb)
                    z_shared = ed_vae.reparam(mu, logv)
            # train critic
            for _ in range(N_CRITIC):
                z = torch.randn(B, SEQ_LEN, NOISE_DIM, device=DEVICE)
                z_pr = torch.randn(B, Z_PRIV, device=DEVICE)
                fake = hyb_gen(z, z_shared, db, z_pr)
                r = hyb_crit(xb, db); f = hyb_crit(fake.detach(), db)
                wass = f.mean() - r.mean()
                gp = gradient_penalty(hyb_crit, xb, fake.detach(), db)
                d_loss = wass + LAMBDA_GP*gp
                d_opt.zero_grad(); d_loss.backward()
                nn.utils.clip_grad_norm_(hyb_crit.parameters(), 1.0)
                d_opt.step(); dsum += d_loss.item()
            # domain head (GRL) — encourage domain‑invariant z
            dom_logits = hyb_dom(z_shared, lambd=1.0)
            dom_loss = nn.CrossEntropyLoss()(dom_logits, db)
            dom_opt.zero_grad(); dom_loss.backward(); dom_opt.step(); ds += dom_loss.item()
            # generator
            z = torch.randn(B, SEQ_LEN, NOISE_DIM, device=DEVICE)
            z_pr = torch.randn(B, Z_PRIV, device=DEVICE)
            fake = hyb_gen(z, z_shared, db, z_pr)
            g_loss = -hyb_crit(fake, db).mean()
            g_opt.zero_grad(); g_loss.backward()
            nn.utils.clip_grad_norm_(hyb_gen.parameters(), 1.0)
            g_opt.step(); gsum += g_loss.item()
        if ep%5==0 or ep==1:
            print(f"[HYBRID] {ep}/{epochs}  D={dsum/len(jloader):.3f}  G={gsum/len(jloader):.3f}  Dom={ds/len(jloader):.3f}")

train_hybrid(EPOCHS_HYBRID)


In [ ]:

# ----------------
# Evaluation: ACF / PSD / DTW + Forecast utility
# ----------------
def acf(seq, max_lag):
    x = np.asarray(seq).ravel()
    x = (x - x.mean())/(x.std()+1e-8)
    out = [1.0]
    for k in range(1, max_lag+1):
        out.append(np.corrcoef(x[:-k], x[k:])[0,1])
    return np.array(out)

def psd(seq):
    p = np.abs(np.fft.rfft(np.asarray(seq).ravel()))**2
    return p/(p.sum()+1e-8)

def dtw_dist(a, b):
    a = np.asarray(a).ravel(); b = np.asarray(b).ravel()
    n,m = len(a), len(b)
    D = np.full((n+1,m+1), np.inf); D[0,0]=0.0
    for i in range(1,n+1):
        for j in range(1,m+1):
            cost = abs(a[i-1]-b[j-1])
            D[i,j] = cost + min(D[i-1,j], D[i,j-1], D[i-1,j-1])
    return D[n,m]

def sample_fake(gen, domain, n=128):
    gen.eval()
    with torch.no_grad():
        z = torch.randn(n, SEQ_LEN, NOISE_DIM, device=DEVICE)
        z_sh = torch.randn(n, Z_SHARED, device=DEVICE)
        z_pr = torch.randn(n, Z_PRIV, device=DEVICE)
        d = torch.full((n,), int(domain), dtype=torch.long, device=DEVICE)
        fake = gen(z, z_sh, d, z_pr).cpu().numpy().squeeze(-1)
    return fake

real_ed = ed_seq[:256]
fake_ed_b = sample_fake(ed_gen_b, 0, min(256, len(real_ed)))
fake_ed_h = sample_fake(hyb_gen, 0, min(256, len(real_ed)))

if len(fin_seq)>0 and fin_gen_b is not None:
    real_fin = fin_seq[:256]
    fake_fin_b = sample_fake(fin_gen_b, 1, min(256, len(real_fin)))
    fake_fin_h = sample_fake(hyb_gen, 1, min(256, len(real_fin)))
else:
    real_fin = fake_fin_b = fake_fin_h = None

def quality(real, fake, name):
    max_lag = 20
    r_acf = np.mean([acf(s, max_lag) for s in real], axis=0)
    f_acf = np.mean([acf(s, max_lag) for s in fake], axis=0)
    acf_delta = float(np.linalg.norm(r_acf - f_acf))

    r_psd = np.mean([psd(s) for s in real], axis=0)
    f_psd = np.mean([psd(s) for s in fake], axis=0)
    psd_delta = float(np.linalg.norm(r_psd - f_psd))

    # DTW: mean nearest neighbor distance (limit for speed)
    ds = []
    R = real[:64]; F = fake[:32]
    for f in F:
        ds.append(min(dtw_dist(f, r) for r in R))
    dtw_mean = float(np.mean(ds))

    print(f"[{name}]  ACFΔ={acf_delta:.3f}  PSDΔ={psd_delta:.3f}  DTW̄={dtw_mean:.3f}")
    return {"acf": acf_delta, "psd": psd_delta, "dtw": dtw_mean}

print("\nQuality — ED")
m_ed_b = quality(real_ed, fake_ed_b, "ED baseline")
m_ed_h = quality(real_ed, fake_ed_h, "ED hybrid")

if real_fin is not None:
    print("\nQuality — Finance")
    m_fin_b = quality(real_fin, fake_fin_b, "FIN baseline")
    m_fin_h = quality(real_fin, fake_fin_h, "FIN hybrid")


In [ ]:

# Forecasting utility (tiny LSTM, 10 epochs)
class TinyLSTM(nn.Module):
    def __init__(self, h=96):
        super().__init__()
        self.l = nn.LSTM(1, h, batch_first=True)
        self.f = nn.Linear(h, 1)
    def forward(self, x):
        y,_ = self.l(x); return self.f(y[:,-1,:])

def to_sup(seqs, horizon=1):
    X=[];Y=[]
    for s in seqs:
        X.append(s[:-horizon]); Y.append(s[-horizon:][0])
    X=np.array(X)[...,None]; Y=np.array(Y)[...,None]
    return torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.float32)

def forecast_eval(train_set, test_set, tag):
    Xtr,Ytr = to_sup(train_set); Xte,Yte = to_sup(test_set)
    net = TinyLSTM(LSTM_H).to(DEVICE)
    opt = optim.Adam(net.parameters(), lr=1e-3)
    crit = nn.MSELoss()
    bs=64
    for ep in range(10):
        idx = torch.randperm(len(Xtr))
        for i in range(0, len(Xtr), bs):
            xb = Xtr[idx[i:i+bs]].to(DEVICE); yb = Ytr[idx[i:i+bs]].to(DEVICE)
            loss = crit(net(xb), yb); opt.zero_grad(); loss.backward(); opt.step()
    net.eval()
    with torch.no_grad():
        pred = net(Xte.to(DEVICE)).cpu().numpy().ravel()
        true = Yte.numpy().ravel()
    rmse = float(np.sqrt(np.mean((pred-true)**2)))
    smape = float(100*np.mean(2*np.abs(pred-true)/(np.abs(pred)+np.abs(true)+1e-8)))
    print(f"[Forecast {tag}] RMSE={rmse:.4f}  SMAPE={smape:.2f}%")
    return {"rmse": rmse, "smape": smape}

# ED utility
split_e = int(0.8*len(ed_seq))
ed_train_sup, ed_test_sup = ed_seq[:split_e], ed_seq[split_e:]
ed_aug = np.concatenate([ed_train_sup, fake_ed_h[:len(ed_train_sup)]], axis=0)

print("\nUtility — ED")
u_ed_real = forecast_eval(ed_train_sup, ed_test_sup, "ED real-only")
u_ed_aug  = forecast_eval(ed_aug,       ed_test_sup, "ED real+hybrid")
u_ed_syn  = forecast_eval(fake_ed_h[:len(ed_train_sup)], ed_test_sup, "ED synth-only")

# Finance utility (if present & enough)
if real_fin is not None and len(fin_seq) > 200:
    split_f = int(0.8*len(fin_seq))
    fin_train_sup, fin_test_sup = fin_seq[:split_f], fin_seq[split_f:]
    fin_aug = np.concatenate([fin_train_sup, fake_fin_h[:len(fin_train_sup)]], axis=0)
    print("\nUtility — Finance")
    u_fin_real = forecast_eval(fin_train_sup, fin_test_sup, "FIN real-only")
    u_fin_aug  = forecast_eval(fin_aug,       fin_test_sup, "FIN real+hybrid")
    u_fin_syn  = forecast_eval(fake_fin_h[:len(fin_train_sup)], fin_test_sup, "FIN synth-only")


In [ ]:

summary = {
    "quality_ED": {"baseline": m_ed_b, "hybrid": m_ed_h},
    "utility_ED": {"real": u_ed_real, "real+hybrid": u_ed_aug, "synth_only": u_ed_syn},
}
try:
    summary["quality_FIN"] = {"baseline": m_fin_b, "hybrid": m_fin_h}
except NameError:
    pass
print("\n=== Summary (lower is better for all metrics) ===")
print(json.dumps(summary, indent=2))
print("\nExpectation: Hybrid should reduce ACF/PSD deltas and DTW vs baseline, and improve ED forecasting when augmenting with hybrid synthetic data.")
